<a href="https://colab.research.google.com/github/DanielDialektico/rag_agentes_langchain_curso/blob/main/notebooks/langchain_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://dialektico.com/wp-content/uploads/2023/03/MiniLogoW4.png" alt="Dialéktico Logo" />

Este pequeño tutorial pertenece al curso de RAG y agentes con LangChain al que puedes acceder mediante la siguiente URL: https://www.youtube.com/playlist?list=PLlWTv9_GeWd32stuEMWpYOnxiVxnXaU6q

Sigue los videos del curso para recibir instrucciones y contexto sobre la ejecución de este Notebook.

<br>

# Se instalan e importan las librerías

In [ ]:
!pip install langchain==0.3.20
!pip install langchain_deepseek==0.1.2

In [ ]:
import os
import warnings
from langchain_deepseek import ChatDeepSeek
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate
from google.colab import userdata

warnings.filterwarnings('ignore')

## Se añade valor de API key mediante un secreto

In [ ]:
# Se añade la API key como variable de ambiente desde un secreto en Colab.
os.environ["DEEPSEEK_API_KEY"] = userdata.get('DEEPSEEK_API_KEY')

## Se declara el modelo a utilizar

In [ ]:
# Se define el modelo y añaden valores de parámetros.
model = ChatDeepSeek(
      model="deepseek-chat",
      temperature=0,
      max_tokens=100
      )

<br>

# Estructura de mensajes

​En LangChain, los mensajes se clasifican en diferentes tipos para estructurar y gestionar eficazmente las interacciones con los modelos. Los elementos son los siguientes:

* **SystemMessage:** se utiliza para proporcionar instrucciones o contexto al modelo de inteligencia artificial, orientando su comportamiento en la conversación. Corresponde a un rol de sistema (system).

* **HumanMessage:** El HumanMessage representa la entrada proporcionada por el usuario durante la interacción. Corresponde a un rol de usuario (user).

* **AIMessage:** El AIMessage corresponde a la respuesta generada por el modelo de inteligencia artificial. Corresponde a un rol de asistente (assistant).

* **AIMessageChunk**: se utiliza para stremear respuestas (en flujo). Corresponde a un rol de asistente (assistant).

* **ToolMessage**: se utiliza para observar los resultados de llamadas de herramientas. Tiene un rol de herramienta (tool).

Más información: https://python.langchain.com/docs/concepts/messages/

In [ ]:
# Se imprime la estructura del mensaje.
response = model.invoke('¡Hola!')
response

In [ ]:
# Se imprime únicamente la respuesta.
response.content

In [ ]:
# Se añaden instrucciones a nivel system, y se inserta un prompt.
messages = [
    SystemMessage("Eres un asistente virtual llamado Dialectibot, saluda cordialmente diciendo tu nombre."),
    HumanMessage("¡Hola!"),
]

In [ ]:
# Se obtiene la respuesta del modelo.
response = model.invoke(messages)
response

In [ ]:
# Se imprime únicamente la respuesta.
response.content

Más información sobre runnables en LangChain: https://python.langchain.com/docs/concepts/runnables/

<br>

# Plantillas de prompts

Los templates de prompts ayudan a traducir la entrada del usuario y los parámetros en instrucciones para un modelo de lenguaje. Esto se puede utilizar para guiar la respuesta del modelo, ayudándolo a comprender el contexto y generar una salida relevante y coherente.

Los templates de prompts reciben como entrada un diccionario, donde cada clave representa una variable dentro del template de prompt que debe completarse.

## Zero-Shot prompting

El Zero-Shot Prompting es una técnica en la que se le da solo una instrucción al modelo para generar una respuesta.

En LanghChain existen templates de prompts que se pueden utilizar para dar formato a una sola cadena de texto.

In [ ]:
# Se crea una plantilla de prompts.
prompt_template = PromptTemplate.from_template("Dime el nombre de un libro famoso de {genre}. Solo dime el nombre, no des más información.")

# Se genera un prompt utilizando la plantilla
prompt = prompt_template.invoke({"genre": "fantasía"})
prompt

In [ ]:
response = model.invoke(prompt)
response.content

Se pueden generar plantillas para los prompts y las instrucciones a nivel system:

In [ ]:
# Se crea una plantilla de prompts.
prompt_template = ChatPromptTemplate([
    ("system", "Eres un asistente experto en libros del género {genre}. Responde a la pregunta solo diciendo el nombre del libro que se te pide, no des más información."),
    ("user", "Dime el nombre de un libro famoso de {genre}.")
])

prompt = prompt_template.invoke({"genre": "ciencia ficción"})
prompt

In [ ]:
response = model.invoke(prompt)
response.content

In [ ]:
# Se crea una plantilla de prompts.
prompt_template = ChatPromptTemplate([
    SystemMessage("Eres un asistente experto en libros del género {genre}. Responde a la pregunta solo diciendo el nombre del libro que se te pide, no des más información."),
    HumanMessage("Dime el nombre de un libro famoso de {genre}."),
])

prompt = prompt_template.invoke({"genre": "ciencia ficción"})
prompt

In [ ]:
response = model.invoke(prompt)
response.content

Más información en: https://python.langchain.com/docs/concepts/prompt_templates/

<br>

## Few shot prompting y One-Shot prompting

El Few-Shot Prompting es una técnica en la que se proporcionan al modelo de lenguaje algunos ejemplos dentro del prompt para ayudarlo a entender el patrón de la tarea antes de generar una respuesta. El One Shot prompting consiste en pasar un solo ejemplo.

Existen plantillas de LangChain que ayudan a la organización de este tipo de prompts.

In [ ]:
# Lista de ejemplos de few-shot
examples = [
    {"word": "happy", "answer": "Dialektico está triste"},
    {"word": "big", "answer": "Dialektico es pequeño"},
    {"word": "weak", "answer": "Dialektico es fuerte"},
]

examples

In [ ]:
example_prompt = PromptTemplate.from_template("Palabra Clave: {word}\nRespuesta esperada: {answer}")
example_prompt

In [ ]:
print(example_prompt.invoke(examples[0]).to_string())

In [ ]:
# Creación del FewShotPromptTemplate
prompt  = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""Ejemplos:""",
    suffix="De acuerdo a los ejemplos, responde a la siguiente Palabra Clave: {input}",
    input_variables=["input"],
    example_separator="\n"
)

prompt = prompt.invoke({"input": "Flaco"}).to_string()
prompt

In [ ]:
response = model.invoke(prompt)
response.content

<br>

# Streams en respuestas

Los LLM pueden tardar varios segundos en generar una respuesta completa a una consulta. Esto es mucho más lento que el umbral de ~200-300 ms en el que una aplicación se siente receptiva para el usuario final.

La estrategia clave para hacer que la aplicación se sienta más receptiva es mostrar un progreso intermedio, es decir, transmitir la salida del modelo token por token.

LangChain permite esto mediante un "stream" de las respuestas:

In [ ]:
chunks = []
for chunk in model.stream("¿Cuál es el nombre completo de Da Vinci?"):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

Más información: https://python.langchain.com/docs/how_to/streaming/

<br>

In [ ]:
# Dialektico Machine learning practices © 2025 by Daniel Antonio García Escobar
# is licensed under CC BY-NC 4.0. To view a copy of this license,
# visit https://creativecommons.org/licenses/by-nc/4.0/

# Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International
# Public License